In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import pymongo
from pprint import pprint

# --- Connect to SQLite ---
conn = sqlite3.connect("../../sql/openfda_base_updated.db")
cursor = conn.cursor()
#11090837

In [2]:
# count entries in drug table where safetyreportid is 11090837
cursor.execute("SELECT COUNT(*) FROM drug WHERE safetyreportid = 11090837")
count = cursor.fetchone()[0]
print(f"Number of entries in drug table with safetyreportid 11090837: {count}")

Number of entries in drug table with safetyreportid 11090837: 561


In [3]:
### count reactions for SELECT COUNT(*) FROM drug WHERE safetyreportid = 11090837

cursor.execute("SELECT COUNT(*) FROM reaction WHERE safetyreportid = 11090837")
count = cursor.fetchone()[0]
print(f"Number of entries in drug table with safetyreportid 11090837: {count}")

Number of entries in drug table with safetyreportid 11090837: 8


In [8]:
### print all reactions for SELECT COUNT(*) FROM drug WHERE safetyreportid = 11090837
cursor.execute("SELECT * FROM reaction WHERE safetyreportid = 11090837")
reactions = cursor.fetchall()
for reaction in reactions:
    print(reaction[1:])

(11090837, 'Coma', 26.1, 1)
(11090837, 'Gastrooesophageal reflux disease', 26.1, 1)
(11090837, 'Pneumonia aspiration', 26.1, 1)
(11090837, 'Somnolence', 26.1, 1)
(11090837, 'Incorrect route of product administration', 26.1, 1)
(11090837, 'Off label use', 26.1, 1)
(11090837, 'Incorrect dose administered by product', 26.1, 1)
(11090837, 'Underdose', 26.1, 1)


In [28]:
cursor.execute("SELECT * FROM patient_optional WHERE safetyreportid = 11090837")
patient_optional = cursor.fetchall()
for patient in patient_optional:
    print(patient)

(11090837, 5, 64, 801, 2, 69.0)


In [16]:
cursor.execute("SELECT COUNT(*) FROM report_duplicate WHERE safetyreportid = 11090837")
count = cursor.fetchone()[0]
print(f"Number of entries in report_duplicate table with safetyreportid 11090837: {count}")

Number of entries in report_duplicate table with safetyreportid 11090837: 105


In [4]:
# count entries in drug table where safetyreportid is 11090837
cursor.execute("SELECT medicinalproduct FROM drug WHERE safetyreportid = 11090837")
count = cursor.fetchall()

unique_medicinal_products = set()
for row in count:
    unique_medicinal_products.add(row[0])
print(count)

[('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('MOMETASONE FUROATE',), ('TIZANIDINE HYDROCHLORIDE',), ('TIZANIDINE HYDROCHLORIDE',), ('FUROSEMIDE',), ('PREDNISONE',), ('WARFARIN',), ('BIFIDOBACTERIUM LONGUM\\LACTOBACILLUS ACIDOPHILUS',), ('POTASSIUM CHLORIDE',), ('POTASSIUM CHLORIDE',), ('POTASSIUM CHLORIDE',), ('POTASSIUM CHLORIDE',), ('ATIVAN',), ('VITAMIN B',), ('BACLOFEN',), ('CELLCEPT',), ('CYMBALTA',), ('CYMBAL

In [5]:
print(len(unique_medicinal_products))
print(unique_medicinal_products)

106
{'RIBOFLAVIN 5^-PHOSPHATE SODIUM ANHYDROUS', 'INOSITOL', 'ASCORBIC ACID', 'MOMETASONE FUROATE', 'VITAMIN B12', 'AMINOBENZOIC ACID', 'DEXPANTHENOL', 'CALCIUM GLUCONATE', 'ALKA-SELTZER', 'SULFAMETHOXAZOLE\\TRIMETHOPRIM', 'PREDNISONE', 'IMODIUM', 'WARFARIN', 'GLICLAZIDE', 'RETINOL', 'LYRICA', 'ZANTAC', 'DOCUSATE SODIUM', 'SULFADIAZINE\\TRIMETHOPRIM', 'THIAMINE', 'TRIMETHOPRIM', 'BIFIDOBACTERIUM LONGUM\\LACTOBACILLUS ACIDOPHILUS', 'LACTOBACILLUS RHAMNOSUS', 'DILPACIMAB', 'SULFAMETHOXAZOLE', 'RANITIDINE', 'CREATININE', 'NICOTINAMIDE', 'CREATINE', 'THIAMINE HYDROCHLORIDE', 'TIZANIDINE', 'NIACINAMIDE', 'DOCONEXENT', 'NIACINAMIDE\\PYRIDOXINE HYDROCHLORIDE\\RIBOFLAVIN\\THIAMINE HYDROCHLORIDE', 'LOPERAMIDE', 'TIZANIDINE HYDROCHLORIDE', 'MELATONIN', 'CALCIUM CARBONATE', 'HYDROMORPHONE HYDROCHLORIDE', 'SENNOSIDES A AND B', 'ALENDRONATE SODIUM', 'DULOXETINE HYDROCHLORIDE', 'PYRIDOXINE HYDROCHLORIDE', 'ALKA-SELTZER HEARTBURN RELIEFCHEWS', 'VITAMIN B', 'LACTOBACILLUS CASEI', 'ZOPICLONE', 'POTASSI

In [12]:
# get the drug ids for the drugs form safetyreportid 11090837
cursor.execute("SELECT medicinalproduct, id FROM drug WHERE safetyreportid = 11090837")
drug_ids = cursor.fetchall()
ids = []
for drug_id in drug_ids:
    ids.append(drug_id)
# for drug_id in drug_ids:
    # print(drug_id[0])

In [13]:
print(ids[:10])

[('MOMETASONE FUROATE', 116519), ('MOMETASONE FUROATE', 116520), ('MOMETASONE FUROATE', 116521), ('MOMETASONE FUROATE', 116522), ('MOMETASONE FUROATE', 116523), ('MOMETASONE FUROATE', 116524), ('MOMETASONE FUROATE', 116525), ('MOMETASONE FUROATE', 116526), ('MOMETASONE FUROATE', 116527), ('MOMETASONE FUROATE', 116528)]


In [14]:
print(len(ids))

561


In [ ]:
dupa = []

def get_ids():
    for product in list(unique_medicinal_products):
        # print(product)
        ids = set()
        cursor.execute("SELECT id FROM drug WHERE safetyreportid = 11090837 AND medicinalproduct = ?", (product,))
        drug_ids = cursor.fetchall()
        for drug_id in drug_ids:
            # print(drug_id[0])
            ids.add(drug_id[0])
        ids = list(ids)
        dupa.append((product, ids))

get_ids()
print(len(dupa))

106


In [22]:
multi = []
for drug in dupa:
    if len(drug[1]) > 1:
        multi.append(drug)

print(len(multi))


36


In [25]:
def get_meta():

    for drug in multi:
        # ile = 
        name = drug[0]
        ids = drug[1]

        dupa = set()

        for id in ids:
            query = f"""
            SELECT *
            FROM drug_openfda
            WHERE drug_id = '{id}'
            """
            results = conn.execute(query).fetchall()
            # print(type(results))
            # print(results[0][2:])
            dupa.add(results[0][2:])

        if len(dupa) > 1:
            print(f"Drug: {name}")
        else:
            print(f"Drug: {name} - no duplicates")
            continue


get_meta()        

Drug: ASCORBIC ACID - no duplicates
Drug: MOMETASONE FUROATE - no duplicates
Drug: VITAMIN B12 - no duplicates
Drug: CALCIUM GLUCONATE - no duplicates
Drug: SULFAMETHOXAZOLE\TRIMETHOPRIM - no duplicates
Drug: IMODIUM - no duplicates
Drug: SULFADIAZINE\TRIMETHOPRIM - no duplicates
Drug: LACTOBACILLUS RHAMNOSUS - no duplicates
Drug: THIAMINE HYDROCHLORIDE - no duplicates
Drug: TIZANIDINE - no duplicates
Drug: TIZANIDINE HYDROCHLORIDE - no duplicates
Drug: CALCIUM CARBONATE - no duplicates
Drug: HYDROMORPHONE HYDROCHLORIDE - no duplicates
Drug: SENNOSIDES A AND B - no duplicates
Drug: ALENDRONATE SODIUM - no duplicates
Drug: DULOXETINE HYDROCHLORIDE - no duplicates
Drug: PYRIDOXINE HYDROCHLORIDE - no duplicates
Drug: ALKA-SELTZER HEARTBURN RELIEFCHEWS - no duplicates
Drug: ZOPICLONE - no duplicates
Drug: POTASSIUM CHLORIDE - no duplicates
Drug: LORATADINE - no duplicates
Drug: INULIN - no duplicates
Drug: VITAMIN D NOS - no duplicates
Drug: HUMAN IMMUNOGLOBULIN G - no duplicates
Drug: CYM

In [33]:
# Check how many times each drug id from ids is not present in the drug_activesubstance table
missing_active_substances = []

for drug_id in ids:
    cursor.execute(f"SELECT COUNT(*) FROM drug_activesubstance WHERE drug_id = {drug_id}")
    count = cursor.fetchone()[0]
    if count == 0:
        missing_active_substances.append(drug_id)

print(f"Number of drugs without active substances: {len(missing_active_substances)}")
print(f"Percentage of drugs without active substances: {len(missing_active_substances) / len(ids) * 100:.2f}%")

# Get the medicinal products for these missing active substances
if missing_active_substances:
    # Get a sample of up to 10 missing drugs to display
    sample_size = min(10, len(missing_active_substances))
    sample_ids = missing_active_substances[:sample_size]
    
    placeholders = ', '.join('?' for _ in sample_ids)
    query = f"SELECT id, medicinalproduct FROM drug WHERE id IN ({placeholders})"
    
    cursor.execute(query, sample_ids)
    sample_drugs = cursor.fetchall()
    
    print("\nSample of drugs without active substances:")
    for drug_id, medicinal_product in sample_drugs:
        print(f"ID: {drug_id}, Medicinal Product: {medicinal_product}")

Number of drugs without active substances: 0
Percentage of drugs without active substances: 0.00%


In [34]:
missing_fda = []

for drug_id in ids:
    cursor.execute(f"SELECT COUNT(*) FROM drug_openfda WHERE drug_id = {drug_id}")
    count = cursor.fetchone()[0]
    if count == 0:
        missing_fda.append(drug_id)

print(f"Number of drugs without active substances: {len(missing_fda)}")
print(f"Percentage of drugs without active substances: {len(missing_fda) / len(ids) * 100:.2f}%")

# Get the medicinal products for these missing active substances
if missing_fda:
    # Get a sample of up to 10 missing drugs to display
    sample_size = min(10, len(missing_fda))
    sample_ids = missing_fda[:sample_size]
    
    placeholders = ', '.join('?' for _ in sample_ids)
    query = f"SELECT id, medicinalproduct FROM drug WHERE id IN ({placeholders})"
    
    cursor.execute(query, sample_ids)
    sample_drugs = cursor.fetchall()
    
    print("\nSample of drugs without active substances:")
    for drug_id, medicinal_product in sample_drugs:
        print(f"ID: {drug_id}, Medicinal Product: {medicinal_product}")

Number of drugs without active substances: 0
Percentage of drugs without active substances: 0.00%
